In [1]:
from python_on_whales import DockerClient, docker
import uuid

docker_clients = []
for i in range(1):
    worker_id = uuid.uuid4().hex
    docker_client = DockerClient(
        compose_project_name=worker_id,
        compose_files=["client-compose.yml"],
    )
    docker_clients.append(docker_client)



In [3]:
for docker_client in docker_clients:
    docker_client.compose.up(
        detach=True,
        build=True,
    )

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 121B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 121B done
#2 WARN: LegacyKeyValueFormat: "ENV key=value" should be used instead of legacy "ENV key value" format (line 7)
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 0.0s

#4 [app-product internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [app-account 1/4] FROM docker.io/library/golang:latest
#5 DONE 0.0s

#6 [app-account internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.0s

#7 [app-product internal] load build context
#7 transferring context: 5.08kB 0.0s done
#7 DONE 0.0s

#8 [app-product 2/4] WORKDIR /workdir
#8 CACHED

#9 [app-account internal] load build context
#9 transferring context: 335B 0.0s done
#9 DON

KeyboardInterrupt: 

In [3]:
docker.image.list()

[python_on_whales.Image(id='sha256:b4201f2066453', tags=['tiangolo/uvicorn-gunicorn-fastapi:latest']),
 python_on_whales.Image(id='sha256:0db8fd8d8896a', tags=[]),
 python_on_whales.Image(id='sha256:5f6fa28313b81', tags=[]),
 python_on_whales.Image(id='sha256:43283e931cec8', tags=[]),
 python_on_whales.Image(id='sha256:a61b645b609b7', tags=['golang:latest']),
 python_on_whales.Image(id='sha256:cf8d6ecc3574a', tags=['muazhari/autocode-example-app-account:latest']),
 python_on_whales.Image(id='sha256:1c0fd857deda9', tags=['muazhari/autocode-example-app-product:latest']),
 python_on_whales.Image(id='sha256:28e8ac7aafcf5', tags=[])]

In [5]:
for docker_client in docker_clients:
    docker_client.compose.down()

 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Stopping
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Stopping
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Stopped
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Removing
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Stopped
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Removing
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-product-1  Removed
 Container 577f83d86ebd46d59dcdd5c466efb96f-app-account-1  Removed
 Network 577f83d86ebd46d59dcdd5c466efb96f_default  Removing
 Network 577f83d86ebd46d59dcdd5c466efb96f_default  Removed
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Stopping
 Container 9bae1087874f45d885f2d86ae6fe903f-app-account-1  Stopping
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Stopped
 Container 9bae1087874f45d885f2d86ae6fe903f-app-product-1  Removing
 Container 9bae1087874f45d885f2d86ae6fe903f-app-account-1  Stopped
 Co

In [15]:
list(docker_clients[0].compose.ps()[0].network_settings.networks.values())[0].ip_address

[NetworkInspectResult(ipam_config=None, links=None, aliases=['f4425bc7e176405e89ca8af2954caa46-app-account-1', 'app-account'], network_id='00785bea842e5b3820f4f95902ec47281ceb44bba67df60dfdc6bdbfd20763b0', endpoint_id='', gateway='', ip_address='', ip_prefix_length=0, ipv6_gateway='', global_ipv6_address='', global_ipv6_prefix_length=0, mac_address='', driver_options=None)]

In [17]:
for docker_client in docker_clients:
    for container in docker_client.compose.ps():
        A_WORKER_ID = docker_client.client_config.compose_project_name
        B_WORKER_ID = [env for env in container.config.env if "WORKER_ID" in env][0].split("=")[1]
        print(A_WORKER_ID, B_WORKER_ID, A_WORKER_ID == B_WORKER_ID, container.name)

f4425bc7e176405e89ca8af2954caa46 f4425bc7e176405e89ca8af2954caa46 True f4425bc7e176405e89ca8af2954caa46-app-account-1
f4425bc7e176405e89ca8af2954caa46 f4425bc7e176405e89ca8af2954caa46 True f4425bc7e176405e89ca8af2954caa46-app-product-1


In [20]:
container.name.removeprefix("f4425bc7e176405e89ca8af2954caa46-")

'app-product-1'

In [5]:
from httpx import Client

http_client = Client(
    base_url="http://172.18.0.3:8000",
)
http_client.get("/").json()

{'worker_id': 'd7d6146b1c2c4fb8ac4934277a567520', 'app_name': 'app-1'}

In [1]:
import autocode

optimization = autocode.Optimization(
    num_cpus=2,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

2024-08-03 16:01:46,045	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 2.0, 'object_store_memory': 4503869030.0, 'memory': 9007738062.0, 'GPU': 1.0}


 Container 6729c63409334f42936b79a350bb47ec-app-account-1  Stopping
 Container 6729c63409334f42936b79a350bb47ec-app-product-1  Stopping
 Container 6729c63409334f42936b79a350bb47ec-app-account-1  Stopped
 Container 6729c63409334f42936b79a350bb47ec-app-account-1  Removing
 Container 6729c63409334f42936b79a350bb47ec-app-account-1  Removed
 Container 6729c63409334f42936b79a350bb47ec-app-product-1  Stopped
 Container 6729c63409334f42936b79a350bb47ec-app-product-1  Removing
 Container 6729c63409334f42936b79a350bb47ec-app-product-1  Removed
 Network 6729c63409334f42936b79a350bb47ec_default  Removing
 Network 6729c63409334f42936b79a350bb47ec_default  Removed
 Container e0dd85ec075547f680ce8ce90e9bf193-app-product-1  Stopping
 Container e0dd85ec075547f680ce8ce90e9bf193-app-account-1  Stopping
 Container e0dd85ec075547f680ce8ce90e9bf193-app-product-1  Stopped
 Container e0dd85ec075547f680ce8ce90e9bf193-app-product-1  Removing
 Container e0dd85ec075547f680ce8ce90e9bf193-app-product-1  Removed
 Co

Server is healthy.


In [2]:
docker_clients = optimization.deploy(
    compose_files=["./client-compose.yml"],
)

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 101B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 101B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 0.0s

#4 [app-product internal] load .dockerignore
#4 transferring context: 2B done
#4 DONE 0.0s

#5 [app-account internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account 1/4] FROM docker.io/library/golang:latest
#6 DONE 0.0s

#7 [app-product internal] load build context
#7 transferring context: 335B 0.0s done
#7 DONE 0.0s

#8 [app-account internal] load build context
#8 transferring context: 335B 0.0s done
#8 DONE 0.0s

#9 [app-account 3/4] COPY . .
#9 CACHED

#10 [app-account 4/4] RUN go mod tidy
#10 CACHED

#11 [app-product 3/4] COPY . .
#11 CACHED

#12 [app-account

 Network 7ce1dc94ab5d4d5f9769b40e916e1984_default  Creating
 Network 7ce1dc94ab5d4d5f9769b40e916e1984_default  Created
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-product-1  Creating
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-account-1  Creating
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-account-1  Created
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-product-1  Created
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-product-1  Starting
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-account-1  Starting
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-account-1  Started
 Container 7ce1dc94ab5d4d5f9769b40e916e1984-app-product-1  Started
gio: http://localhost:10001: Operation not supported


#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 101B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 101B 0.0s done
#2 DONE 0.0s

#3 [app-account internal] load metadata for docker.io/library/golang:latest
#3 DONE 0.0s

#4 [app-product internal] load .dockerignore
#4 transferring context: 2B 0.0s done
#4 DONE 0.0s

#5 [app-account internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-account 1/4] FROM docker.io/library/golang:latest
#6 DONE 0.0s

#7 [app-account internal] load build context
#7 transferring context: 335B 0.0s done
#7 DONE 0.0s

#8 [app-product internal] load build context
#8 transferring context: 335B 0.0s done
#8 DONE 0.0s

#9 [app-account 3/4] COPY . .
#9 CACHED

#10 [app-account 4/4] RUN go mod tidy
#10 CACHED

#11 [app-product 2/4] WORKDIR /workdir
#11 CACHED

#12

 Network d6fac759a2bc4f90ae36089aad84efec_default  Creating
 Network d6fac759a2bc4f90ae36089aad84efec_default  Created
 Container d6fac759a2bc4f90ae36089aad84efec-app-account-1  Creating
 Container d6fac759a2bc4f90ae36089aad84efec-app-product-1  Creating
 Container d6fac759a2bc4f90ae36089aad84efec-app-product-1  Created
 Container d6fac759a2bc4f90ae36089aad84efec-app-account-1  Created
 Container d6fac759a2bc4f90ae36089aad84efec-app-product-1  Starting
 Container d6fac759a2bc4f90ae36089aad84efec-app-account-1  Starting
 Container d6fac759a2bc4f90ae36089aad84efec-app-account-1  Started
 Container d6fac759a2bc4f90ae36089aad84efec-app-product-1  Started


In [8]:
docker_clients[0].container.list()[0].network_settings.networks

{'1495ece1976645b6b9cf6fa5dffeb511_default': NetworkInspectResult(ipam_config=None, links=None, aliases=['1495ece1976645b6b9cf6fa5dffeb511-app-account-1', 'app-account'], network_id='fa9f642d4782b36dbdc320732f6d02a231160ab35c5c9e1ec743d7994ed7d5fe', endpoint_id='0e2b0971791d338ef765c1f579332e929a86c487251cb1ee54c328c91cfb4fc3', gateway='192.168.160.1', ip_address='192.168.160.2', ip_prefix_length=20, ipv6_gateway='', global_ipv6_address='', global_ipv6_prefix_length=0, mac_address='02:42:c0:a8:a0:02', driver_options=None)}

In [3]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(input: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    output: Dict[str, Any] = {
        "F": input[0].objectives + input[1].objectives,
    }

    return output


In [4]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

AttributeError: 'dict' object has no attribute 'id'